In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import json
#import utm as utm

#*********basic restaurant data
restaurant = pd.read_csv("restaurants.csv")

# restaurant.dropna(
#     axis=0,
#     how='any',
#     thresh=None,
#     subset=None,
#     inplace=True
# )
restaurant.dropna()

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(restaurant, 
                        lat="LATITUDE", 
                        lon="LONGITUDE", 
                        hover_name="NAME", 
                        hover_data=["PHYSICALADDRESS", "PHYSICALCITY"],
                        # color="Listed",
                        # color_continuous_scale=color_scale,
                        zoom=9, 
                        height=300)


# fig = go.Figure(go.Scattermapbox())

#lat_lon = utm.to_latlon(508009.1730000004, 5446008.925249999, 10, 'U')
#print(lat_lon)

# fig = px.scatter_mapbox(restaurant, lat="Lat", lon="Long", hover_name="Address", hover_data=["Address", "Listed"],
#                         color_discrete_sequence=["fuchsia"], zoom=3, height=300)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import json
from types import SimpleNamespace
#*********basic transit data
#transit_json = open('dataset/transit.json')

with open('dataset/transit.json') as your_file:
    your_dict = json.load(your_file)
    your_file.seek(0)
    your_root = json.load(your_file, object_hook= lambda x: 
                                        SimpleNamespace(**x))
type(your_root[2])
# transit = json.load(transit_json)
# for i in transit[2]:
#     print(i)

In [ ]:
from scripts.model import TransitStopRaw
from scripts.data_processing import parse_to_raw_stop_data, convert_to_latlon

transit_stop_array_raw = []

parse_to_raw_stop_data(transit_stop_array_raw)

transit_stop_array = []
lat_array = []
lon_array = []

convert_to_latlon(transit_stop_array_raw, transit_stop_array, lat_array, lon_array)


for i in range(5):
    transitStopEle = transit_stop_array[i]
    print(f"Stop {i+1}:")
    print(f"id: {transitStopEle.id}")
    print(f"lat: {transitStopEle.lat}")
    print(f"lon: {transitStopEle.lon}")
    print(f"location: {transitStopEle.location}")
    print(f"lonarr_ele_: {lon_array[i]}")
    print(f"latarr_ele_: {lat_array[i]}")
    print("----------------")



In [ ]:
import geopandas as gpd
from shapely.geometry import Point

import pandas as pd
import numpy as np

df1 = gpd.GeoDataFrame(
    {'geometry': [Point(xy) for xy in zip(lon_array, lat_array)]},
    crs="EPSG:4326")
df2 = gpd.GeoDataFrame(
    {'geometry': [Point(xy) for xy in zip(lon_array, lat_array)]},
    crs="EPSG:4326")

df1['x'] = df1.geometry.x
df1['y'] = df1.geometry.y
df2['x'] = df2.geometry.x
df2['y'] = df2.geometry.y

# Merge the two GeoDataFrames on the x and y columns
merged = pd.merge(df1[['x', 'y']], df2[['x', 'y']], on=['x', 'y'])

# Calculate the correlation between the two sets of coordinates
correlation = np.corrcoef(merged.x, merged.y)[0, 1]
print(f"The correlation between the two GeoDataFrames is: {correlation}")
#
# df1.to_file("../data/processed/data.shp", driver="ESRI Shapefile")

# w = ps.weights.Queen.from_dataframe(df1)
# cij = ps_explore.spatial_covariance(df1['geometry'], df2['geometry'], w, transform='r')

# ps_explore.plot_cis(cij)

